In [7]:
import json, os, random
from moviepy.editor import VideoFileClip
from IPython.display import Video, display


def draw_rectangle(box):
    def draw(frame):
        """Draw a rectangle in the frame"""
        fr = frame.copy()
        bottom = int(box['y1'])
        top = int(bottom + box['height'])
        left = int(box['x1'])
        right = int(left + box['width'])
        fr[top-5:top+5, left:right] = 255 - fr[top-5:top+5, left:right]
        fr[bottom-5:bottom+5, left:right] = 255 - fr[bottom-5:bottom+5, left:right]
        fr[bottom:top, left-5:left+5] = 255 - fr[bottom:top, left-5:left+5]
        fr[bottom:top, right-5:right+5] = 255 - fr[bottom:top, right-5:right+5]
        return fr
    return draw

def draw_keypoints(keypoints, fps, size = 5):
    def draw(get_frame, t):
        """Draw keypoints in the frame"""
        fr = get_frame(t).copy()
        keypoints_t: list[float] = keypoints[min(int(t * fps), len(keypoints)-1)]['keypoints']
        it = iter(keypoints_t)
        for x, y, conf in zip(it, it, it):
            #if conf > 0.5:
            fr[int(y)-size:int(y)+size:,int(x)-size:int(x)+size] = 255-fr[int(y)-size:int(y)+size:,int(x)-size:int(x)+size]
        return fr
    return draw

def group_kds(kds):
    '''Groups keypoint data objects that belong to same frame'''
    grouped = [[]]
    for kd in kds:
        added = False
        for g in grouped:
            if not added and (not g or g[-1]['image_id'] != kd['image_id']):
                g.append(kd)
                added = True
        if not added:
            grouped.append([kd])
    return grouped

def update_clip(clip):
    rec_clip = VideoFileClip(clip+".mp4")
    with open(clip+"_signer.json") as signerf:
        signer = json.load(signerf)
    with open(clip+"_ap.json") as apf:
        ap = json.load(apf)
    signers = group_kds(ap)
    for s in signers:
        rec_clip = rec_clip.fl(draw_keypoints(s, rec_clip.fps, 7))
    final_clip = rec_clip.fl_image(draw_rectangle(signer['roi']))
    final_clip.write_videofile("../data/temp/temp.mp4", codec="libx264", fps=15, verbose=False, logger=None)
    with open(clip+".json") as dataf:
        data = json.load(dataf)
    return data

path = "../data/cuts/"
cuts = [(path+vid+'/'+cut[:-4]) for vid in os.listdir(path) for cut in os.listdir(path + vid) if cut.endswith(".mp4")]

In [8]:
clip = random.choice(cuts)

data = update_clip(clip)

print("Video: {}".format(clip))
print("Label: {}".format(data['label']))
display(Video("../data/temp/temp.mp4", height=300))

Video: ../data/cuts/noticias-en-lengua-de-senas-argentina-resumen-semanal-28032021/121
Label: lograr igualdad en la competencia. chau.


In [7]:
error = input("Crop error in video {}? y/[n]".format(clip))
if error == 'y':
    with open("../data/errors.csv", 'a') as errors_f:
        errors_f.write(clip[:-1] + ',')
if os.path.isfile("../data/errors.csv"):
    with open("../data/errors.csv", 'r') as errors_f:
        for f in errors_f.read().split(','):
            print(f)

../data/cuts/como-hacer-una-huerta-urbana-en-casa-episodio-8-en-lengua-de-senas-argentina-lsa/4



In [8]:
clip = path + input("Clip:")

data = update_clip(clip)

print("Video: {}".format(clip))
print("Label: {}".format(data['label']))
display(Video("../data/temp/temp.mp4", height=300))

Video: ../data/cuts/como-hacer-una-huerta-urbana-en-casa-episodio-8-en-lengua-de-senas-argentina-lsa/4
Label: el alimento disponible.
